In [28]:
import pandas
import numpy
import os
import math
import urllib.request
import zipfile
import timeit
import copy
from datetime import datetime

BOLD = '\033[1m'
END = '\033[0m'
BROWN = "\033[0;33m"
BROWN2 = "\x1b[38;5;3m"

#Marchenko rodion Data Analysis prakt №4

print(BOLD+"MARCHENKO RODION FB-23 PRAKT №4.1 \"Using Pandas and Numpy for numerical data Analysis\"")
print("="*86+END)


#This function loads the initial dataset files from the Internet as a zip archive and decompresses it to the computer, if not already present
def InitializeLoad():
	#Creating temporary directory if it does not exist
	CurrentDirectory = os.getcwd()
	if (os.getcwd()[-13:] != "/DataLab4-tmp"):
		try:
			os.chdir(CurrentDirectory+"/DataLab4-tmp")
		except:
			try:
				os.makedirs(CurrentDirectory+"/DataLab4-tmp")
				print("»» Created tmp directory /DataLab4-tmp.")
			except FileExistsError:
				# directory already exists
				pass
			os.chdir(CurrentDirectory+"/DataLab4-tmp") #GO TO DATASET DIRECTORY!!!

	print("»» The current working directory is: "+os.getcwd()+".\n")


	#Loading data, unzipping and renaming the CSV
	CWD = os.getcwd()
	AvailableFiles = os.listdir(CWD)
	CurrentTime = datetime.now().strftime("%d-%m-%y-%H-%M-%S")

	Filename = CWD+"/Household-electric-power-consumption--"+CurrentTime+".zip"
	UnzippedFileName = "household_power_consumption"
	URL = "https://archive.ics.uci.edu/static/public/235/individual+household+electric+power+consumption.zip"

	if any(UnzippedFileName+".csv" in path for path in AvailableFiles):
		pass
	else:
		try:
			urllib.request.urlretrieve(URL,Filename)
			with zipfile.ZipFile(Filename, 'r') as zip_ref:
				zip_ref.extractall(CWD)
			os.rename(CWD+"/"+UnzippedFileName+".txt", CWD+"/"+UnzippedFileName+".csv")

			print("\t» Downloaded Household electric power consumption dataset from the internet.")
		except:
			print("\t» ERROR: Failed to load the dataset.")
	return CWD


#This function reads datasets as pandas frame and numpy array
def ReadDatasetFromCSV(WorkDir):
	headers = ['Date', 'Time', 'GlobalActivePower', 'GlobalReactivePower', 'Voltage', 'GlobalIntensity', 'SubMetering1', 'SubMetering2', 'SubMetering3'] #CSV table headers
	print("»» "+BOLD+"Loading data from file:"+END)
	for path in os.listdir(WorkDir):
		FullPath = os.path.join(WorkDir, path)
		if (os.path.isfile(FullPath) and path[-4:] == ".csv"): #Find the first existing CSV file
			print(path)
			df = pandas.read_csv(FullPath, header = 1, names = headers, delimiter = ";") #Load into pandas dataframe
			nparray = numpy.genfromtxt(FullPath, delimiter = ";", missing_values="?", skip_header=1, dtype="str") #Load into numpy array
			break

	return [df,nparray]



#"MAIN" FUNCTION:
CWD = InitializeLoad()
FullDataSet = ReadDatasetFromCSV(CWD)

PdDataSet = FullDataSet[0] #PANDAS DATAFRAME VERSION OF THE DATASET
NpDataSet = FullDataSet[1] #NUMPY ARRAY VERSION OF THE DATASET

TstPdDataSet = copy.copy(PdDataSet) #PANDAS DATAFRAME VERSION OF THE DATASET FOR TIME PROFILING
TstNpDataSet = copy.copy(NpDataSet) #NUMPY ARRAY VERSION OF THE DATASET FOR TIME PROFILING



MARCHENKO RODION FB-23 PRAKT №4.1 "Using Pandas and Numpy for numerical data Analysis"
»» The current working directory is: /home/rodion/DataLab4-tmp.

»» Loading data from file:
household_power_consumption.csv


/tmp/ipykernel_47485/1205458444.py:73: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(FullPath, header = 1, names = headers, delimiter = ";")


In [29]:

#This function performs some preprocessing on Pandas Dataframe
def PandasPrepareData(Dataset):
	Dataset = Dataset.dropna(axis=0) #Drop empty rows
	Dataset.insert(2, "DateTime", Dataset["Date"]+" "+Dataset["Time"]) #Insert a numpy-compatible date-time column
	Dataset["DateTime"] = pandas.to_datetime(Dataset["DateTime"], format="%d/%m/%Y %H:%M:%S") #Cast column to datetime64
	Dataset = Dataset.reset_index(drop=True) #Reset global index without preserving the old one
	Dataset = Dataset.astype({"GlobalActivePower":"float32", "GlobalReactivePower":"float32", "Voltage":"float32", "GlobalIntensity":"float32", "SubMetering1":"float32", "SubMetering2":"float32", "SubMetering3":"float32"}) #Convert columns to floats for further use in conditions
	return Dataset

PdDataSet = PandasPrepareData(PdDataSet)
t1 =  timeit.timeit(stmt="PandasPrepareData(TstPdDataSet)", number=3, globals=globals())
print("\n"+"="*84)
print(PdDataSet)
print(BROWN+BOLD+"Сер. Час підготовки даних з допомогою Pandas: "+END+str(t1)+END,"\n")



/tmp/ipykernel_47485/4134689651.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset["DateTime"] = pandas.to_datetime(Dataset["DateTime"], format="%d/%m/%Y %H:%M:%S") #Cast column to datetime64
/tmp/ipykernel_47485/4134689651.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset["DateTime"] = pandas.to_datetime(Dataset["DateTime"], format="%d/%m/%Y %H:%M:%S") #Cast column to datetime64
/tmp/ipykernel_47485/4134689651.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic


               Date      Time            DateTime  GlobalActivePower  \
0        16/12/2006  17:25:00 2006-12-16 17:25:00              5.360   
1        16/12/2006  17:26:00 2006-12-16 17:26:00              5.374   
2        16/12/2006  17:27:00 2006-12-16 17:27:00              5.388   
3        16/12/2006  17:28:00 2006-12-16 17:28:00              3.666   
4        16/12/2006  17:29:00 2006-12-16 17:29:00              3.520   
...             ...       ...                 ...                ...   
2049274  26/11/2010  20:58:00 2010-11-26 20:58:00              0.946   
2049275  26/11/2010  20:59:00 2010-11-26 20:59:00              0.944   
2049276  26/11/2010  21:00:00 2010-11-26 21:00:00              0.938   
2049277  26/11/2010  21:01:00 2010-11-26 21:01:00              0.934   
2049278  26/11/2010  21:02:00 2010-11-26 21:02:00              0.932   

         GlobalReactivePower     Voltage  GlobalIntensity  SubMetering1  \
0                      0.436  233.630005             23.0  

In [30]:

#This function performs some preprocessing on Numpy Array
def NumpyPrepareData(Array):
	Array = numpy.where((Array == '?') | (Array == ''),numpy.nan, Array) #replace dataset`s empty markers with numpy.nan
	NumSubArray = Array[:,2:].astype("float32") #Number only subarray, because you can`t compare string to NaN
	Array = Array[~numpy.isnan(NumSubArray).any(axis=1)] #Drop empty rows

	#CONVERSION TO STRUCTURED ARRAY:
	dtype = numpy.dtype([("Date","object"),("Time","object"),("GlobalActivePower","float32"),("GlobalReactivePower","float32"),("Voltage","float32"),("GlobalIntensity","float32"),("SubMetering1","float32"),("SubMetering2","float32"),("SubMetering3","float32")]) #Define the NumPy data type of each column
	tuples = numpy.rec.fromarrays(Array.transpose(), dtype=dtype) #Turn the original np.array into a set of tuples, representing each row of the array
	SArray = numpy.array(tuples, dtype=dtype) #Convert to NumPy structured array
	return SArray

NpDataSet = NumpyPrepareData(NpDataSet)
t2 =  timeit.timeit(stmt="NumpyPrepareData(TstNpDataSet)", number=3, globals=globals())
print("\n"+"="*84)
print(NpDataSet)
print(NpDataSet.shape)
print(BROWN+BOLD+"Сер. Час підготовки даних з допомогою NumPy: "+END+str(t2)+END,"\n")

print("\n"+"="*84)




[('16/12/2006', '17:24:00', 4.216, 0.418, 234.84, 18.4, 0., 1., 17.)
 ('16/12/2006', '17:25:00', 5.36 , 0.436, 233.63, 23. , 0., 1., 16.)
 ('16/12/2006', '17:26:00', 5.374, 0.498, 233.29, 23. , 0., 2., 17.) ...
 ('26/11/2010', '21:00:00', 0.938, 0.   , 239.82,  3.8, 0., 0.,  0.)
 ('26/11/2010', '21:01:00', 0.934, 0.   , 239.7 ,  3.8, 0., 0.,  0.)
 ('26/11/2010', '21:02:00', 0.932, 0.   , 239.55,  3.8, 0., 0.,  0.)]
(2049280,)
Сер. Час підготовки даних з допомогою NumPy: 34.535258568001154 




In [31]:
#Завдання №1 з використанням Pandas:


print(BOLD+"\nВсі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт:"+END)

PdActivePower = PdDataSet[PdDataSet["GlobalActivePower"] > 5]
#Pandas
t1 =  timeit.timeit(stmt="PdDataSet[PdDataSet['GlobalActivePower'] > 5]", number=9, globals=globals())
#Profiling

print(BROWN+"Pandas:\n"+END,PdActivePower)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t1)+END,"\n")




Всі домогосподарства, у яких загальна активна споживана потужність перевищує 5 кВт:
Pandas:
                Date      Time            DateTime  GlobalActivePower  \
0        16/12/2006  17:25:00 2006-12-16 17:25:00              5.360   
1        16/12/2006  17:26:00 2006-12-16 17:26:00              5.374   
2        16/12/2006  17:27:00 2006-12-16 17:27:00              5.388   
10       16/12/2006  17:35:00 2006-12-16 17:35:00              5.412   
11       16/12/2006  17:36:00 2006-12-16 17:36:00              5.224   
...             ...       ...                 ...                ...   
2043376  22/11/2010  18:40:00 2010-11-22 18:40:00              5.408   
2043377  22/11/2010  18:41:00 2010-11-22 18:41:00              5.528   
2045606  24/11/2010  07:50:00 2010-11-24 07:50:00              5.172   
2045607  24/11/2010  07:51:00 2010-11-24 07:51:00              5.750   
2047017  25/11/2010  07:21:00 2010-11-25 07:21:00              5.074   

         GlobalReactivePower     Voltage 

In [32]:
#Завдання №1 з використанням NumPy:


NpActivePower = NpDataSet[NpDataSet["GlobalActivePower"] > 5]
#NumPy
t2 =  timeit.timeit(stmt="NpDataSet[numpy.where(NpDataSet['GlobalActivePower'] > 5)]", number=9, globals=globals())
#Profiling

print(BROWN+"NumPy:\n"+END,NpActivePower,"\n")
print(NpActivePower.shape,"rows.")
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t2)+END)

      

NumPy:
 [('16/12/2006', '17:25:00', 5.36 , 0.436, 233.63, 23. , 0.,  1., 16.)
 ('16/12/2006', '17:26:00', 5.374, 0.498, 233.29, 23. , 0.,  2., 17.)
 ('16/12/2006', '17:27:00', 5.388, 0.502, 233.74, 23. , 0.,  1., 17.) ...
 ('24/11/2010', '07:50:00', 5.172, 0.05 , 235.18, 22. , 0., 38., 17.)
 ('24/11/2010', '07:51:00', 5.75 , 0.   , 234.4 , 24.6, 0., 39., 17.)
 ('25/11/2010', '07:21:00', 5.074, 0.24 , 238.55, 21.4, 1.,  2., 18.)] 

(17547,) rows.
Сер. Час виконання: 0.08118037399981404


In [33]:
#Завдання №2 з використанням Pandas:


print(BOLD+"\n\nВсі домогосподарства, у яких вольтаж перевищує 235 В:"+END)

PdVoltage = PdDataSet[PdDataSet["Voltage"] > 235]
#Pandas
t1 =  timeit.timeit(stmt="PdDataSet[PdDataSet['Voltage'] > 235]", number=9, globals=globals())
#Profiling

print(BROWN+"Pandas:\n"+END,PdVoltage)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t1)+END,"\n")





Всі домогосподарства, у яких вольтаж перевищує 235 В:
Pandas:
                Date      Time            DateTime  GlobalActivePower  \
3        16/12/2006  17:28:00 2006-12-16 17:28:00              3.666   
4        16/12/2006  17:29:00 2006-12-16 17:29:00              3.520   
5        16/12/2006  17:30:00 2006-12-16 17:30:00              3.702   
6        16/12/2006  17:31:00 2006-12-16 17:31:00              3.700   
13       16/12/2006  17:38:00 2006-12-16 17:38:00              4.054   
...             ...       ...                 ...                ...   
2049274  26/11/2010  20:58:00 2010-11-26 20:58:00              0.946   
2049275  26/11/2010  20:59:00 2010-11-26 20:59:00              0.944   
2049276  26/11/2010  21:00:00 2010-11-26 21:00:00              0.938   
2049277  26/11/2010  21:01:00 2010-11-26 21:01:00              0.934   
2049278  26/11/2010  21:02:00 2010-11-26 21:02:00              0.932   

         GlobalReactivePower     Voltage  GlobalIntensity  SubMetering

In [34]:
#Завдання №2 з використанням NumPy:


NpVoltage = NpDataSet[numpy.where(NpDataSet["Voltage"] > 235)]
#NumPy
t2 =  timeit.timeit(stmt="NpDataSet[numpy.where(NpDataSet['Voltage'] > 235)]", number=9, globals=globals())
#Profiling

print(BROWN+"NumPy:\n"+END,NpVoltage,"\n")
print(NpVoltage.shape,"rows.")
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t2)+END)



NumPy:
 [('16/12/2006', '17:28:00', 3.666, 0.528, 235.68, 15.8, 0., 1., 17.)
 ('16/12/2006', '17:29:00', 3.52 , 0.522, 235.02, 15. , 0., 2., 17.)
 ('16/12/2006', '17:30:00', 3.702, 0.52 , 235.09, 15.8, 0., 1., 17.) ...
 ('26/11/2010', '21:00:00', 0.938, 0.   , 239.82,  3.8, 0., 0.,  0.)
 ('26/11/2010', '21:01:00', 0.934, 0.   , 239.7 ,  3.8, 0., 0.,  0.)
 ('26/11/2010', '21:02:00', 0.932, 0.   , 239.55,  3.8, 0., 0.,  0.)] 

(1952491,) rows.
Сер. Час виконання: 1.6000912300005439


In [35]:
#Завдання №3 з використанням Pandas:


print(BOLD+"\n\nВсі домогосподарства, у яких сила струму лежить в межах 19-20 та у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер:"+END)

PdCurrentAndPower = PdDataSet[(PdDataSet["GlobalIntensity"] <= 20) & (PdDataSet["GlobalIntensity"] >= 19) & (PdDataSet["SubMetering2"] > PdDataSet["SubMetering3"])]
#Pandas
t1 =  timeit.timeit(stmt="PdDataSet[(PdDataSet['GlobalIntensity'] <= 20) & (PdDataSet['GlobalIntensity'] >= 19) & (PdDataSet['SubMetering2'] > PdDataSet['SubMetering3'])]", number=9, globals=globals())
#Profiling

print(BROWN+"Pandas:\n"+END,PdCurrentAndPower)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t1)+END,"\n")





Всі домогосподарства, у яких сила струму лежить в межах 19-20 та у яких пральна машина та холодильних споживають більше, ніж бойлер та кондиціонер:
Pandas:
                Date      Time            DateTime  GlobalActivePower  \
44       16/12/2006  18:09:00 2006-12-16 18:09:00              4.464   
459      17/12/2006  01:04:00 2006-12-17 01:04:00              4.582   
463      17/12/2006  01:08:00 2006-12-17 01:08:00              4.618   
474      17/12/2006  01:19:00 2006-12-17 01:19:00              4.636   
475      17/12/2006  01:20:00 2006-12-17 01:20:00              4.634   
...             ...       ...                 ...                ...   
2045609  24/11/2010  07:53:00 2010-11-24 07:53:00              4.666   
2045610  24/11/2010  07:54:00 2010-11-24 07:54:00              4.694   
2045611  24/11/2010  07:55:00 2010-11-24 07:55:00              4.602   
2045612  24/11/2010  07:56:00 2010-11-24 07:56:00              4.536   
2045613  24/11/2010  07:57:00 2010-11-24 07:57:00

In [36]:
#Завдання №3 з використанням NumPy:


NpCurrentAndPower = NpDataSet[numpy.where((NpDataSet["GlobalIntensity"] <= 20) & (NpDataSet["GlobalIntensity"] >= 19) & (NpDataSet["SubMetering2"] > NpDataSet["SubMetering3"]))]
#NumPy
t2 =  timeit.timeit(stmt="NpDataSet[numpy.where((NpDataSet['GlobalIntensity'] <= 20) & (NpDataSet['GlobalIntensity'] >= 19) & (NpDataSet['SubMetering2'] > NpDataSet['SubMetering3']))]", number=9, globals=globals())

print(BROWN+"NumPy:\n"+END,NpCurrentAndPower,"\n")
print(NpCurrentAndPower.shape,"rows.")
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t2)+END)



NumPy:
 [('16/12/2006', '18:09:00', 4.464, 0.136, 234.66, 19. , 0., 37., 16.)
 ('17/12/2006', '01:04:00', 4.582, 0.258, 238.08, 19.6, 0., 13.,  0.)
 ('17/12/2006', '01:08:00', 4.618, 0.104, 239.61, 19.6, 0., 27.,  0.) ...
 ('24/11/2010', '07:55:00', 4.602, 0.   , 237.08, 19.4, 0., 40., 17.)
 ('24/11/2010', '07:56:00', 4.536, 0.   , 237.03, 19. , 0., 39., 17.)
 ('24/11/2010', '07:57:00', 4.626, 0.   , 236.78, 19.4, 0., 39., 17.)] 

(2509,) rows.
Сер. Час виконання: 0.1355831249984476


In [37]:
#Завдання №4 з використанням Pandas:


print(BOLD+"\n\nСередні величини усіх 3-х груп споживання електричної енергії для 500000 випадкових домогосподарств (без повторів елементів вибірки):"+END)


PdRandomRows = PdDataSet.sample(500000, replace=False)


def PdMeans(RandomRows, DataSet):
	SubMetering1Mean = RandomRows["SubMetering1"].mean()
	SubMetering2Mean = RandomRows["SubMetering2"].mean()
	SubMetering3Mean = RandomRows["SubMetering3"].mean()
	return [SubMetering1Mean, SubMetering2Mean, SubMetering3Mean]

means = PdMeans(PdRandomRows, PdDataSet)
#Pandas
t1 = timeit.timeit(stmt="PdMeans(PdRandomRows, PdDataSet)", number=9, globals=globals())
#Profiling

print(BROWN+"\nPandas:"+END)
print("Середні значення для трьох груп споживання: "+BOLD+str(means[0])+", "+str(means[1])+", "+str(means[2])+END)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t1)+END,"\n")





Середні величини усіх 3-х груп споживання електричної енергії для 500000 випадкових домогосподарств (без повторів елементів вибірки):

Pandas:
Середні значення для трьох груп споживання: 1.129112, 1.29499, 6.463414
Сер. Час виконання: 0.010875205000047572 



In [38]:
#Завдання №4 з використанням NumPy:

RandomRows = numpy.random.choice(NpDataSet, replace=True, size=500000) #Select random indexes

def NpMeans(RandomRows, DataSet):
	SubMetering1Mean = RandomRows["SubMetering1"].mean()
	SubMetering2Mean = RandomRows["SubMetering2"].mean()
	SubMetering3Mean = RandomRows["SubMetering3"].mean()
	return [SubMetering1Mean, SubMetering2Mean, SubMetering3Mean]

means = NpMeans(RandomRows, NpDataSet)
#NumPy
t2 = timeit.timeit(stmt="NpMeans(RandomRows, NpDataSet)", number=9, globals=globals())
#Profiling

print(BROWN+"NumPy:"+END)
print("Середні значення для трьох груп споживання: "+BOLD+str(means[0])+", "+str(means[1])+", "+str(means[2])+END)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t2)+END,"\n")



NumPy:
Середні значення для трьох груп споживання: 1.12775, 1.29389, 6.460998
Сер. Час виконання: 0.02752230600162875 



In [39]:
#Завдання №5 з використанням Pandas:


print(BOLD+"\n\nДомогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, у яких основне споживання електроенергії припадає на пральну машину, сушарку, холодильник та освітлення:"+END)

PdAfter1800 = PdDataSet[(PdDataSet["DateTime"].dt.time >= pandas.Timestamp("18:00:00").time()) & (PdDataSet["SubMetering2"] > PdDataSet["SubMetering1"]) & (PdDataSet["SubMetering2"] > PdDataSet["SubMetering3"]) & ((PdDataSet["GlobalActivePower"]**2 + PdDataSet["GlobalReactivePower"]**2)**0.5 > 6)]
#Pandas
t1 =  timeit.timeit(stmt="PdDataSet[(PdDataSet['DateTime'].dt.time >= pandas.Timestamp('18:00:00').time()) & (PdDataSet['SubMetering2'] > PdDataSet['SubMetering1']) & (PdDataSet['SubMetering2'] > PdDataSet['SubMetering3']) & ((PdDataSet['GlobalActivePower']**2 + PdDataSet['GlobalReactivePower']**2)**0.5 > 6)]", number=9, globals=globals())
#Profiling

print(BROWN2+BOLD+"Pandas:\n"+END+END,PdAfter1800)
#print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t1)+END,"\n")

PdAfter1800FirstHalf = PdAfter1800.iloc[0:math.floor(PdAfter1800.shape[0]/2):3]
PdAfter1800SecondHalf = PdAfter1800.iloc[math.floor(PdAfter1800.shape[0]/2)::4]
#Pandas
t11 = timeit.timeit(stmt="PdAfter1800.iloc[0:math.floor(PdAfter1800.shape[0]/2):3]", number=9, globals=globals())
t12 = timeit.timeit(stmt="PdAfter1800.iloc[math.floor(PdAfter1800.shape[0]/2)::4]", number=9, globals=globals())
#Profiling

print(BROWN+"Кожен третій результат із першої половини:"+END)
print(PdAfter1800FirstHalf)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t11)+END,"\n")
print(BROWN+"Кожен четвертий результат із другої половини:"+END)
print(PdAfter1800SecondHalf)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t12)+END,"\n")





Домогосподарства, які після 18-00 споживають понад 6 кВт за хвилину в середньому, у яких основне споживання електроенергії припадає на пральну машину, сушарку, холодильник та освітлення:
Pandas:
                Date      Time            DateTime  GlobalActivePower  \
40       16/12/2006  18:05:00 2006-12-16 18:05:00              6.052   
41       16/12/2006  18:06:00 2006-12-16 18:06:00              6.752   
42       16/12/2006  18:07:00 2006-12-16 18:07:00              6.474   
43       16/12/2006  18:08:00 2006-12-16 18:08:00              6.308   
70       16/12/2006  18:35:00 2006-12-16 18:35:00              6.072   
...             ...       ...                 ...                ...   
2040501  20/11/2010  18:45:00 2010-11-20 18:45:00              6.564   
2040502  20/11/2010  18:46:00 2010-11-20 18:46:00              6.438   
2040503  20/11/2010  18:47:00 2010-11-20 18:47:00              6.000   
2040505  20/11/2010  18:49:00 2010-11-20 18:49:00              6.210   
2040508  2

In [40]:
#Завдання №5 з використанням NumPy:


NpAfter1800 = NpDataSet[numpy.where((NpDataSet["SubMetering2"] > NpDataSet["SubMetering1"]) & (NpDataSet["SubMetering2"] > NpDataSet["SubMetering3"]) & ((NpDataSet["GlobalActivePower"]**2 + NpDataSet["GlobalReactivePower"]**2)**0.5 > 6) & (NpDataSet["Time"] >= "18:00:00"))]
#NumPy
t2 = timeit.timeit(stmt="NpDataSet[numpy.where((NpDataSet['SubMetering2'] > NpDataSet['SubMetering1']) & (NpDataSet['SubMetering2'] > NpDataSet['SubMetering3']) & ((NpDataSet['GlobalActivePower']**2 + NpDataSet['GlobalReactivePower']**2)**0.5 > 6) & (NpDataSet['Time'] >= '18:00:00'))]", number=9, globals=globals())
#Profiling

print(BROWN2+BOLD+"\nNumPy:\n"+END+END,NpAfter1800,"\n")
print(NpAfter1800.shape)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t2)+END,"\n")

NpAfter1800FirstHalf = NpAfter1800[0:math.floor(NpAfter1800.shape[0]/2):3]
NpAfter1800SecondHalf = NpAfter1800[math.floor(NpAfter1800.shape[0]/2)::4]
#Pandas
t21 = timeit.timeit(stmt="NpAfter1800[0:math.floor(NpAfter1800.shape[0]/2):3]", number=9, globals=globals())
t22 = timeit.timeit(stmt="NpAfter1800[math.floor(NpAfter1800.shape[0]/2)::4]", number=9, globals=globals())
#Profiling

print(BROWN+"Кожен третій результат із першої половини:"+END)
print(NpAfter1800FirstHalf[:10])
print(NpAfter1800FirstHalf.shape)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t21)+END,"\n")
print(BROWN+"Кожен четвертий результат із другої половини:"+END)
print(NpAfter1800SecondHalf[:10])
print(NpAfter1800SecondHalf.shape)
print(BROWN+BOLD+"Сер. Час виконання: "+END+str(t22)+END,"\n")






NumPy:
 [('16/12/2006', '18:05:00', 6.052, 0.192, 232.93, 26.2,  0., 37., 17.)
 ('16/12/2006', '18:06:00', 6.752, 0.186, 232.12, 29. ,  0., 36., 17.)
 ('16/12/2006', '18:07:00', 6.474, 0.144, 231.85, 27.8,  0., 37., 16.) ...
 ('20/11/2010', '18:47:00', 6.   , 0.246, 229.5 , 26.4,  9., 40., 16.)
 ('20/11/2010', '18:49:00', 6.21 , 0.174, 228.82, 27.4, 21., 34., 17.)
 ('20/11/2010', '18:52:00', 6.364, 0.126, 229.38, 28.2, 17., 34., 16.)] 

(1072,)
Сер. Час виконання: 0.540849872999388 

Кожен третій результат із першої половини:
[('16/12/2006', '18:05:00', 6.052, 0.192, 232.93, 26.2,  0., 37., 17.)
 ('16/12/2006', '18:08:00', 6.308, 0.116, 232.25, 27. ,  0., 36., 17.)
 ('28/12/2006', '20:58:00', 6.386, 0.374, 236.63, 27. ,  1., 36., 17.)
 ('28/12/2006', '21:02:00', 8.088, 0.262, 235.5 , 34.4,  1., 72., 17.)
 ('28/12/2006', '21:05:00', 7.23 , 0.152, 235.22, 30.6,  1., 73., 17.)
 ('28/12/2006', '21:08:00', 7.352, 0.   , 235.45, 31.2,  1., 73., 17.)
 ('28/12/2006', '21:11:00', 9.048, 0.   ,